In [74]:
import tushare as ts
import pandas as pd
import numpy as np

In [75]:
data = ts.get_hist_data('600497')
m =data

In [76]:
#OBV线（成交量多空比率净额法）
obv = m.volume*((m.close-m.low)-(m.high-m.close))/(m.high-m.close)
obv = obv.fillna(0.0)
obv.apply(np.cumsum)
print('obv:',len(obv))

obv: 695


In [77]:
#MACD
ema12 = m.close.copy()               
ema26 = ema12.copy()              
for k in range(1,len(m)): ema12[k] = (ema12[k-1]*11 + ema12[k]*2)/13.0
for k in range(1,len(m)): ema26[k] = (ema26[k-1]*25 + ema26[k]*2)/27.0
dif = ema12-ema26
dea = dif.copy()
for k in range(1,len(dea)): dea[k] = (dea[k-1]*8 + dea[k]*2)/10.0
macd = dif-dea

In [78]:
#rsi指标（统计近段时间收盘涨数和跌数来判断买卖意向）
n=6
rsi = m.p_change.copy()
rsi.apply(np.sign)
rsi = rsi.rolling(center=False,window=n).mean()

In [100]:
#AR BR CR 指标          
refVa = m.open[1:].values           #Ar
refVb = m.close[:-1].values          #BR
refVc = (2*m.close[:-1] + m.high[:-1] +m.low[:-1]).values /4     #CR
ref = np.median([refVa, refVb, refVc])
a = m.high[1:] - ref
b = ref - m.low[1:]

c = a.rolling(26).sum()
d = b.rolling(26).sum()
cr = c/d

In [95]:
#kdj 指标
n = 5
Ln =  m.low.rolling(n).min()
Hn =  m.high.rolling(n).max()
rsv = (m.close-Ln)/(Hn-Ln)
para1 = para2 = 1.0/3

K = rsv.copy()
K = K.fillna(0.0)
for k in range(1,len(K)): K[k] = K[k-1]*(1-para1) + K[k]*para1
D = K.copy()
for k in range(1,len(D)): D[k] = D[k-1]*(1-para2) + D[k]*para2

In [103]:
#cci 顺势指标
n= 10
ma = m.close.rolling(n).mean()
md = (ma-m.close).abs().rolling(n).mean()
cci = ((m.close+m.high+m.low)/3 -ma)/md/0.015

In [112]:
#DMI指标
n=12
dm1 = m.high.rolling(2).apply(lambda d:max(d[1]-d[0],0.0))[1:]
dm2 = m.high.rolling(2).apply(lambda d:max(d[0]-d[1],0.0))[1:]

dm1[dm1<dm2] = 0
dm2[dm2<dm1] = 0

tr = (m.high-m.low).abs().values[1:], m.high[1:]-m.close[:-1].abs().values, m.low[1:]-m.close[:-1].abs().values
tr = np.minimum(np.minimum(tr[0], tr[1]) ,tr[2])

dm1 = dm1.rolling(n).mean()     
dm2 = dm2.rolling(n).mean()       
tr  = tr.rolling(n).mean()

di1 = dm1/tr        
di2 = dm2/tr

dx = (di1-di2).abs()/(di1+di2)
adx = dx.rolling(n).mean()

C:\Python\Python36\Lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Python\Python36\Lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  after removing the cwd from sys.path.


In [117]:
#boll指标
n=26
ma = m.close.rolling(n).mean()
md = m.close.rolling(n).std()
up = ma + 2*md
dn = ma - 2*md